Verify the accuracy, completeness, and reliability of source data. Show your results in a SQL or Python output.

-- The accuracy and completeness and reliability is decided based on the source, here excel, csv and json. Accuracy is decided by making sure we have all cell filled with valid data, no nulls, correct data form, no outliers. Completeness can be checked by checking for duplicates, continued primary key, and so on.

In [1]:
!pip install pandas
!pip install xlrd

     ---------------------------------------- 11.1/11.1 MB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 12.9/12.9 MB 1.4 MB/s eta 0:00:00
     -------------------------------------- 509.2/509.2 KB 2.9 MB/s eta 0:00:00
     -------------------------------------- 347.8/347.8 KB 3.6 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Python3.7\python.exe -m pip install --upgrade pip' command.


     ---------------------------------------- 96.5/96.5 KB 1.8 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Python3.7\python.exe -m pip install --upgrade pip' command.


In [2]:
#Fetching and reading Customer data  and creating dataframe for the same for python
import pandas as pd
path_customer = "C:\\Users\\Asus\\Desktop\\PEI Assessment\\P_Assessment\\Customer.xls"
data_customer = pd.read_excel(path_customer, sheet_name='atkoe-u250m')
df_customer = pd.DataFrame(data_customer)
print(data_customer.head(5))

   Customer_ID    First     Last  Age Country
0            1   Joseph     Rice   43     USA
1            2     Gary    Moore   71     USA
2            3     John   Walker   44      UK
3            4     Eric   Carter   38      UK
4            5  William  Jackson   58     UAE


In [ ]:
#Fetching Customer data from db table using SQL
Select * from Customer

In [3]:
#Validating data for Customer table using python
print(data_customer.describe())
print(data_customer.info())
print(df_customer.isnull().sum())
print(df_customer.duplicated().sum())

#Looking at the data, there is no discrepancy like null object, age is within range that shows no outliers, and no duplicates but we need to check accuracy of each cell

#Validating data for Customer table using SQL
Select * 
From Customer
where First IS NULL OR Last IS NULL OR Customer_ID IS NULL OR Age IS NULL

Select Customer_ID, COUNT(*)
From Customer
Group By Customer_ID
having count(*)>1

#Since, I can't run these in any db, hence sharing the query

       Customer_ID         Age
count   250.000000  250.000000
mean    125.500000   47.576000
std      72.312977   18.978011
min       1.000000   18.000000
25%      63.250000   29.000000
50%     125.500000   47.000000
75%     187.750000   63.000000
max     250.000000   80.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_ID  250 non-null    int64 
 1   First        250 non-null    object
 2   Last         250 non-null    object
 3   Age          250 non-null    int64 
 4   Country      250 non-null    object
dtypes: int64(2), object(3)
memory usage: 9.9+ KB
None
Customer_ID    0
First          0
Last           0
Age            0
Country        0
dtype: int64
0


In [4]:
#Checking for correct data format for each column
letter_pattern = r'^[A-Za-z]+$'
number_pattern = r'^[0-9]+$'

first_valid = df_customer['First'].str.match(letter_pattern, na=False)
last_valid = df_customer['Last'].str.match(letter_pattern, na=False)
age_valid = df_customer['Age'].apply(lambda x:str(x).isdigit())
country_valid = df_customer['Country'].str.match(letter_pattern, na=False)

valid_mask = first_valid & last_valid & age_valid & country_valid

valid_rows = df_customer.loc[valid_mask, ['Customer_ID', 'First', 'Last', 'Age', 'Country']]

print(valid_rows)

#Using SQL
Select * 
From Customer
where First REGEXP '^[A-Za-z]+$'
AND Last REGEXP '^[A-Za-z]+$'
AND isnumeric([Age])
AND Country REGEXP '^[A-Za-z]+$'

     Customer_ID     First     Last  Age Country
0              1    Joseph     Rice   43     USA
1              2      Gary    Moore   71     USA
2              3      John   Walker   44      UK
3              4      Eric   Carter   38      UK
4              5   William  Jackson   58     UAE
..           ...       ...      ...  ...     ...
245          246    Justin  Stewart   73     USA
246          247      John   Miller   53     USA
247          248    Thomas  Hickman   23      UK
248          249  Patricia   Garcia   19      UK
249          250   Stephen    Jones   22     USA

[238 rows x 5 columns]


In [5]:
#There is a possibility that for same customer to have multiple ids, to avoid such situation we will confirm
duplicates = valid_rows.groupby(['First', 'Last', 'Age', 'Country'])['Customer_ID'].nunique()
duplicates = duplicates[duplicates>1]
if not duplicates.empty:
    print(f"Duplicate combinations found:{duplicates}")
else:
    print("No duplicates found")

#Using SQL
Select first, last, age, country, count(Customer_ID) as ID
from Customer
group by first, last, age, country
having ID > 1

No duplicates found


In [6]:
#To keep consistancy
df_customer = valid_rows
print(df_customer.info())

# the data validation has removed 12 rows with inconsistent data

<class 'pandas.core.frame.DataFrame'>
Index: 238 entries, 0 to 249
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_ID  238 non-null    int64 
 1   First        238 non-null    object
 2   Last         238 non-null    object
 3   Age          238 non-null    int64 
 4   Country      238 non-null    object
dtypes: int64(2), object(3)
memory usage: 11.2+ KB
None


In [7]:
#Validating data for Order Table using python
path_order = "C:\\Users\\Asus\\Desktop\\PEI Assessment\\P_Assessment\\Order.csv"
data_order = pd.read_csv(path_order)
df_order = pd.DataFrame(data_order)
print(data_order.describe()) 
print(data_order.info())  
print(df_order.isnull().sum())  -- non-null data for all columns
print(df_order.duplicated().sum()) -- no duplicates
print(df_order.head(5))

#Validating Using SQL
Select * 
from Order
where Order_ID IS NULL OR Item IS NULL OR Amount IS NULL OR Customer_ID IS NULL

Select Order_ID, Count(*)
from Order
group by Order_ID
having count(*)>1

         Order_ID       Amount  Customer_ID
count  250.000000    250.00000   250.000000
mean   125.500000   2130.00000   130.404000
std     72.312977   3575.43493    69.192711
min      1.000000    200.00000     4.000000
25%     63.250000    300.00000    71.500000
50%    125.500000    400.00000   125.500000
75%    187.750000   1500.00000   190.750000
max    250.000000  12000.00000   250.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Order_ID     250 non-null    int64 
 1   Item         250 non-null    object
 2   Amount       250 non-null    int64 
 3   Customer_ID  250 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 7.9+ KB
None
Order_ID       0
Item           0
Amount         0
Customer_ID    0
dtype: int64
0
   Order_ID      Item  Amount  Customer_ID
0         1  Keyboard     400          139
1         2     Mouse     300 

In [8]:
#Since Order should only be placed for valid customers, that is customers that are already present in Customer table,usually this is taken care during 
#data modelling to make sure Order table is created using reference from Customer table 

Create table Order(
    Item Varchar(100),
    Order_ID INT,
    Constraint Customer_id
      Foreign key (Customer_id)
      References Customer(Customer_id)
      On delete cascade
)

#Still we can check if the customers present in Order table is already existing in Customer table or not

Select o.customer_id
from Order o
where o.customer_id not in (Select customer_id from Customer)

In [9]:
#Validating the same use case using python
valid_order_with_existing_customers = df_order[df_order['Customer_ID'].isin(df_customer['Customer_ID'])]
print(valid_order_with_existing_customers.head(5))
print(valid_order_with_existing_customers.info())

   Order_ID      Item  Amount  Customer_ID
0         1  Keyboard     400          139
1         2     Mouse     300          250
2         3   Monitor   12000          239
3         4  Keyboard     400          153
4         5  Mousepad     250          153
<class 'pandas.core.frame.DataFrame'>
Index: 238 entries, 0 to 249
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Order_ID     238 non-null    int64 
 1   Item         238 non-null    object
 2   Amount       238 non-null    int64 
 3   Customer_ID  238 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 9.3+ KB
None


In [10]:
print(df_order.info())
#this shows that there were orders created from wrong/dummy/incorrect customer details

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Order_ID     250 non-null    int64 
 1   Item         250 non-null    object
 2   Amount       250 non-null    int64 
 3   Customer_ID  250 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 7.9+ KB
None


In [11]:
#We can check the unmatched values like this
import pandas as pd

merged_df = pd.merge(df_order, df_customer, on='Customer_ID', how='left', indicator=True)
unmatched_orders = merged_df[merged_df['_merge'] =='left_only']
unmatched_orders = unmatched_orders[df_order.columns]
print(unmatched_orders)

#SQL
Select o.*
from Order o
left join Customer c on c.customer_id = o.customer_id
where c.customer_id IS NULL

     Order_ID      Item  Amount  Customer_ID
20         21  Keyboard     400          171
60         61  Keyboard     400          118
64         65  Mousepad     250          198
77         78  Mousepad     200          211
102       103   Monitor   12000          242
168       169   DDR RAM    1500          236
169       170   Headset     900          214
200       201  Keyboard     400          109
212       213   Monitor   12000          118
220       221  Keyboard     400          198
224       225  Mousepad     250          113
230       231  Keyboard     400          162


In [14]:
#Cleaning the actual table
valid_orders = pd.merge(df_order, df_customer, on='Customer_ID', how='inner')
valid_orders_only = valid_orders[df_order.columns]
print(valid_orders_only.info())

df_order=valid_orders_only

#Using SQL

Select o.*
from Order o
inner join Customer c on o.customer_id = c.customer_id

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Order_ID     238 non-null    int64 
 1   Item         238 non-null    object
 2   Amount       238 non-null    int64 
 3   Customer_ID  238 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 7.6+ KB
None


In [16]:
#Cleaning Shipping table the same way
path_shipping = "C:\\Users\\Asus\\Desktop\\PEI Assessment\\P_Assessment\\Shipping.json"
data_shipping = pd.read_json(path_shipping)
df_shipping = pd.DataFrame(data_shipping)
print(data_shipping.describe()) 
print(data_shipping.info())  
print(df_shipping.isnull().sum())  -- no nulls
print(df_shipping.duplicated().sum()) -- no duplicates
print(data_shipping.head(5))

       Shipping_ID  Customer_ID
count   250.000000   250.000000
mean    125.500000   120.620000
std      72.312977    73.893848
min       1.000000     1.000000
25%      63.250000    53.250000
50%     125.500000   118.000000
75%     187.750000   187.500000
max     250.000000   248.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Shipping_ID  250 non-null    int64 
 1   Status       250 non-null    object
 2   Customer_ID  250 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 6.0+ KB
None
Shipping_ID    0
Status         0
Customer_ID    0
dtype: int64
0
   Shipping_ID     Status  Customer_ID
0            1    Pending          173
1            2    Pending          155
2            3  Delivered          242
3            4    Pending          223
4            5  Delivered           72


In [18]:
#Only those orders should have customer details who are already present in order table
valid_shipping_with_existing_customer = df_shipping[df_shipping['Customer_ID'].isin(df_order['Customer_ID'])]
print(valid_shipping_with_existing_customer.head(5))
print(valid_shipping_with_existing_customer.info())

    Shipping_ID     Status  Customer_ID
3             4    Pending          223
5             6    Pending           29
7             8  Delivered          193
9            10  Delivered           17
10           11    Pending          157
<class 'pandas.core.frame.DataFrame'>
Index: 139 entries, 3 to 248
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Shipping_ID  139 non-null    int64 
 1   Status       139 non-null    object
 2   Customer_ID  139 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.3+ KB
None


In [25]:
valid_shipping = pd.merge(df_shipping, df_order, on='Customer_ID', how='inner')
valid_shipping_only = valid_shipping[df_shipping.columns]
print(valid_shipping_only.info())
print(df_shipping.info())

df_shipping = valid_shipping_only

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Shipping_ID  218 non-null    int64 
 1   Status       218 non-null    object
 2   Customer_ID  218 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 5.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Shipping_ID  250 non-null    int64 
 1   Status       250 non-null    object
 2   Customer_ID  250 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 6.0+ KB
None


For accuracy and validation i did 
1. Duplication check
2. Data Format check
3. non null check
4. Data continuation check
5. Dependency check
6. Regex check
7. Outlier check

Based on your findings, define and outline the requirements for anticipated datasets, detailing the proposed domain model to support the reporting requirements noted below. 
Based on the dataset and questions and with the first look this looks like Sales data with product and shipping. With the same case in mind the datasets requirement would be
1. Customer Dimension Table - same as it is, couple of columns can be added based on requirement like gender and if we are planning SCD mainatinance then is_active, joining_date, releasing_date and so on
2. Product Dimension Table - Product_id, product_name
3. Orders (Fact Table)
4. Date - for better hierarchial queries and filters

Proposed Domain Model
While working on staging area for processing, cleaning, and validating, snowflake schema with dimension normalized helps is cleaner data which in later stages can go as it is, if 
1. the same data also brings in sub-categories or the granularity is low that is aggregated, so let's say monthly sales or so
2. multi layer drill down is needed like categories,sub-categories, product or region, market, country
3. if dimension table changes a lot, in this case customer data update or in general region market update,
snowflake schema is a better option as it handles normalized dimension.

Meanwhile, I'll go with star schema if
1. high-granularity or flat dimension to reduce complexity is needed
2. No frequent changes
3. Faster queries due to lower joins - best for reporting

All the dimension table mapped to fact table using one id as one to many relationship between fact and dimension table.

Prepare a story with technical specifications for one part of the data model for a data engineer to include technical specifications and transformations. This story should give enough information for a Data Engineer to build table(s) and for a QA engineer to test it. 

Looking at the sales data, some common questions that I would like to answer few question like sales per month per country, or product sales review, or country sales pattern, or product sales pattern throughout the year, or YoY comparisons.

To answer these questions we can have a target table with either processed raw data that is needed to create a query for all these scenarios or we can have stored procedure saved to create tables for let's say montly sales per country,per product, per customer.

Let's look at both the solutions:
I want dimension tables like
1. Customer Dimension table with customer details where customer id is the primary key and everything is non null field, with no duplicates
2. Product Dimension table with item details like item id, item name, price where item id is primary key and every field is non null field with no duplicates
3. Country Dimension table with country id and name where country id is non null field with no duplicates
4. Date Dimension table that stores full date, month, year, date, month-year key, quarter, day of week

to Achieve Target table from which we can execute queries based on filters and conditions:
1. Fact Table with fields from raw data (customer_id (constraint reference with customer dimension table), item_id (constraint regerence with product table), country_id (constraint reference with country table), sale date, quantity, status, last_updated (to have ETL update timestamp)) - This provides a generic table to perform queries.
2. Scenario Specific Table like sales summary monthly which should be created on 5th of every month,
   a. must have sale date between first and last day of previous month,
   b. status must be completed,
   c. join with customer dim, product dim, country dim, date dim with complete date
   d. group by customer id, item id, country id, sale month
   e. aggregate calculation sum(quantity), sum(amount) based on item id and quantity
   f. table created date - for auditing


QA Validation 
QA can test the scenario by
1. Date filtering - Only records from last calendar month included
2. Record Count - Matches the records from source
3. No nulls in Customer, Item or Country
4. No Duplicates
   


In [ ]:
#the total amount spent and the country for the Pending delivery status for each country.
Select c.country, sum(amount) as total
from Customer s
join Order o
on c.customer_id = 0.customer_id
join Shipping s
on c.customer_id = s.customer_id
where status='Pending'
group by c.country

In [ ]:
#the total number of transactions, total quantity sold, and total amount spent for each customer, along with the product details.

Select count(order_id) as total_order, count(item) as total_item, sum(amount) as total_amount, customer_id, item
from Order
group by customer_id

In [ ]:
#the maximum product purchased for each country.
#1. the maximum number of item purchased for each country
SELECT country, item, total_purchases
FROM (
    SELECT 
        c.country,
        o.item,
        COUNT(*) AS total_purchases,
        RANK() OVER (PARTITION BY c.country ORDER BY COUNT(*) DESC) AS rnk
    FROM orders o
    JOIN customers c ON o.customer_id = c.customer_id
    GROUP BY c.country, o.item
) ranked_items
WHERE rnk = 1;

#2. the maximum amount per country
SELECT country, item, total_amount
FROM (
    SELECT 
        c.country,
        o.item,
        SUM(o.amount) AS total_amount,
        RANK() OVER (PARTITION BY c.country ORDER BY SUM(o.amount) DESC) AS rnk
    FROM orders o
    JOIN customers c ON o.customer_id = c.customer_id
    GROUP BY c.country, o.item
) ranked_items
WHERE rnk = 1;

#3. the maximum items per country irrespective of each item
SELECT country, item, total_purchases
FROM (
    SELECT 
        c.country,
        o.item,
        COUNT(*) AS total_purchases,
        RANK() OVER (PARTITION BY c.country ORDER BY COUNT(*) DESC) AS rnk
    FROM orders o
    JOIN customers c ON o.customer_id = c.customer_id
    GROUP BY c.country, o.item
) ranked_items
WHERE rnk = 1;

In [ ]:
#the most purchased product based on the age category less than 30 and above 30.

Select item, 
SUM(CASE when c.age < 30 then 1 else 0 end) as below_30,
SUM(CASE when c.age > 30 then 1 else 0 end) as above_30
from Order o
Join Customer c on c.customer_id = o.customer_id
group by item
limit 1

In [ ]:
#the country that had minimum transactions and sales amount.
With item_count as (
Select c.country, o.Item, Count(*) as total_product,
                                Row_number() OVER (PARTITION BY c.Country ORDER BY COUNT(*) DESC)as rn,
                                SUM(amount) OVER (PARTITION BY c.Country ORDER BY SUM(amount) ASC) AS amount
                                from Customer c
                                Join Order o on c.cutomer_id = o.customer_id
                                Group by c.country, o.Item
                                                  )
Select Country, Item, total_product, amount from item_count limit 1